# pyodbc를 이용하여 hive에 연결

---
##### ODBC(Open DataBase Connectivity)
어떤 DBMS를 사용하는지 관계없이 DBMS 간 호환될 수 있도록 MS에서 만든 표준<br>

##### API
프로그램들이 서로 상호작용하는 것을 도와주는 매개체<br>
__특징1.__ 서버와 DB에 대한 출입구 역할 → 허용된 사용자에게만 접근성 부여<br>
__특징2.__ 어플리케이션과 기기가 데이터를 원활히 주고받을 수 있도록 돕는 역할<br>
__특징3.__ 모든 접속을 표준화 → 기계/운영체제 등과 상관 없이 동일한 엑세스를 가능하게 함. 범용 플러그처럼 작동.<br>

##### DSN(Data Source Name)
DB를 사용하는 어플리케이션에서 DB를 불러올 때 해당 DB를 연결시키기 위해 구분 짓는 이름

In [ ]:
# 참고로 DSN = <연결할DSN> 이런식으로 띄어쓰면 실행 오류 발생
conn = pyodbc.connect("DSN=<연결할DSN>", autocommit = True)

sql = """
select *
from tmp.tmp_test
"""

df = pd.read_sql(sql, conn)
df.head()

___
# python - hive 연결 시 sql 쿼리 문 안에 변수 넣기

In [ ]:
import pyodbc
import pandas as pd

start = 0
end = 50

conn = pyodbc.connect("DSN=CloudHI", autocommit = True)

sql = """
select *
from tmp.tmp_test
"""
# 변하지 않는 부분은 주석처리로 처리하고, 변수로 변동하는 부분만 설정한 변수 넣어주기
sql = sql + """where rnum > """ + a + """ and rnum <= """ + b
df = pd.read_sql(sql, conn)
df.head()

___
# 동적으로 데이터 프레임 생성하며 데이터 넣기

format과 globals 사용<br>
[참고] https://datata29.github.io/data/2021/03/11/datapre1/

In [ ]:
import pyodbc
import pandas as pd

start = ["""0""", """10""","""20"""]
end = [ """10""","""20""","""30"""]
i = 1

conn = pyodbc.connect("DSN=<연결할DSN>", autocommit = True)

for a,b in zip(start, end) :
    sql = """
    select *
    from tmp.tmp_test
    """
    sql = sql + """where rnum > """ + a + """ and rnum <= """ + b
    globals()['df_{}'.format(i)] = pd.read_sql(sql, conn)
    i = i + 1

##### 문자열 이어붙여서 사용하기(문자열 포맷팅 이용x 방법)

In [ ]:
conn = pyodbc.connect("DSN=CloudHI", autocommit = True)

sql = """
select *
from tmp.tmp_test
"""

for i in tqdm(range(2)) :
    # set start row number and end row number
    # these variable should be 'str' type, not 'int' or sth else
    start = str(i*10)
    end = str((i+1) * 10)
    # add start and end set to base sentence
    sql2 = sql + """where rnum > """ + start + """ and rnum <= """ + end
    # use 'globals()' func, then you can set df name dynamically
    globals()['test_{}'.format(i+1)] = pd.read_sql(sql2, conn)

##### 문자열 포맷팅을 이용하는 방법

In [ ]:
conn = pyodbc.connect("DSN=CloudHI", autocommit = True)

for i in tqdm(range(2)) :
    start = i*10
    end = (i+1) * 10
    sql = f"""select * from tmp.tmp_aws_itm_ord_2112 where rnum > {start} and rnum <= {end}"""
    globals()['test_{}'.format(i+1)] = pd.read_sql(sql, conn)

하지만 동적변수는 디버깅에 어려움이 있다는 글을 보게 되었고 이보다는 리스트 안에 넣는 것을 추천한다고 하신 글을 참고하여 아래 코드 생성

In [ ]:
conn = pyodbc.connect("DSN=CloudHI", autocommit = True)

dfs = {}

for i in tqdm(range(loop)) :
    start = i * 10
    end = (i+1) * 10
    sql = f"""select * from tmp.tmp_aws_itm_ord_2111 where rnum > {start} and rnum <= {end}"""
    dfs.append(pd.read_sql(sql, conn))

In [ ]:
# 리스트 안 데이터 프레임 모두 합쳐 최종 데이터 프레임 생성
fnl = pd.concat(dfs, axis = 1, ignore_index = True)
fnl.head()